In [ ]:
import os
import numpy as np
from single_class_svm import create_svm_model, create_svm_model_robust, predict_across_skills

In [ ]:
dir_ = '../Data/stone_pick_random_pixels_big'
files = os.listdir(dir_ + '/groundTruth')

In [ ]:
def get_unique_skills(files):
    unique_skills = set()
    for file in files:
        with open(os.path.join(dir_ + '/groundTruth', file), 'r') as f:
            lines = f.read().splitlines()
        unique_skills.update(lines)
    return unique_skills


def get_start_end_states(skill, files):
    skill_states = []
    other_states = [] 

    for file in files:
        with open(os.path.join(dir_ + '/groundTruth', file), 'r') as f:
            lines = f.read().splitlines()

        pca_feats = np.load(os.path.join(dir_ + '/pca_features', file + '.npy'))
        n_frames = len(pca_feats)
        # assert n_frames == len(lines), f"Mismatch in {file}: {n_frames} feats vs {len(lines)} labels"

        # indices where this skill appears
        skill_indices = [i for i, x in enumerate(lines) if x == skill]
        if not skill_indices:
            # if the skill never occurs, then *all* frames are "other"
            other_states.extend(pca_feats.tolist())
            continue

        # collect start & end feature vectors
        for s in skill_indices:
            skill_states.append(pca_feats[s].tolist())

        # everything else (exclude only starts and ends)
        excluded = set(skill_indices)
        for idx in range(n_frames):
            if idx not in excluded:
                other_states.append(pca_feats[idx].tolist())

    return (
        np.array(skill_states),
        np.array(other_states),
    )

In [ ]:
skills = get_unique_skills(files)
skill_data = {}
for skill in skills:
    skill_states, other_states = get_start_end_states(skill, files)
    skill_data[skill] = {
        'skill_states': skill_states,
        'other_states': other_states
    }

In [ ]:
svm_models = {}

for skill, data in skill_data.items():
    skill_states = data['skill_states']
    other_states = data['other_states']
    combined_other = np.concatenate((skill_states, other_states), axis=0)

    print("Start model for skill", skill)
    svm_models[skill] = create_svm_model_robust(skill_states, combined_other)
    print("=" * 60)

In [ ]:
for skill in skill_data: 
    #Test the start and end models, add up correct vs incorrect vs total 
    start_states = skill_data[skill]['start_states']
    end_states = skill_data[skill]['end_states']

    correct_start = 0 
    correct_end = 0
    total_start = 0
    total_end = 0

    for feat in start_states:
        predictions = predict_across_skills(feat, svm_start_models)
        if predictions['top_skill'] == skill:
            correct_start += 1
        total_start += 1

    for feat in end_states:
        predictions = predict_across_skills(feat, svm_end_models)
        if predictions['top_skill'] == skill:
            correct_end += 1
        total_end += 1

    print(f"Skill: {skill}")
    print(f"Start Model - Correct: {correct_start}, Total: {total_start}")
    print(f"End Model - Correct: {correct_end}, Total: {total_end}")
    print("=" * 60)

# TODO TOMORROW ADD BACK START AND END SKILLS TO EVALUATE THEM 